# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [ ]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from [here](https://drive.google.com/file/d/17oJ9NNbwJ8Wv4Jp-mLFiC59k72Brdyha/view?usp=sharing) and place it in the provided data folder. You will access the data folder with a relative path.
Load `Orders` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [ ]:
# your code here    
orders = pd.read_csv('data/Orders.csv',index_col=0)

orders.head()

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here
#Sub Problem 1
Total_spent = orders.groupby(['CustomerID'])[["amount_spent"]].sum()
Total_spent.head()

In [ ]:
#Sub Problem 2

Preferred_Cust = np.percentile(Total_spent['amount_spent'], 75)
VIP_Cust = np.percentile(Total_spent['amount_spent'], 95)

print(Preferred_Cust,VIP_Cust)

def type_cust(x):
    if x > VIP_Cust:
        return 'VIP'
    elif x > Preferred_Cust:
        return 'Preferred'
    else:
        return 'normal'
    
    
Total_spent['type_cust'] = Total_spent['amount_spent'].apply(type_cust)

Total_spent.head()


In [ ]:
#Sub Problem 3

orders = pd.merge(orders, Total_spent, on='CustomerID')

orders.head()


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [ ]:
# your code here

#currently customer ID's are repeated, so we need a new data frame with country, customer id and customer type, no duplicates
#adding InvoiceNo because apparently I need a value
VIP_Country = orders.groupby(['type_cust','Country','CustomerID'])[['InvoiceNo']].count()

#resetting index to remove multiindex columns
VIP_Country.reset_index(inplace=True)  

VIP_Country


In [ ]:
#selecting only VIP customers:
VIP_Country = VIP_Country.loc[(VIP_Country['type_cust']=='VIP')]

#group by country, count by customerID and sort to see which country has more VIP customers
VIP_Country.groupby(['Country'])[['CustomerID']].count().sort_values(by='CustomerID',ascending=False)



In [ ]:
## Q3: How to identify which country has the most VIP+Preferred Customers combined?


In [ ]:
# your code here

#currently customer ID's are repeated, so we need a new data frame with country, customer id and customer type, no duplicates
#adding InvoiceNo because apparently I need a value
Pref_VIP_Country = orders.groupby(['type_cust','Country','CustomerID'])[['InvoiceNo']].count()

#resetting index to remove multiindex columns
Pref_VIP_Country.reset_index(inplace=True)

Pref_VIP_Country

#selecting only Preferred customers:
Pref_VIP_Country = Pref_VIP_Country.loc[(Pref_VIP_Country['type_cust']=='Preferred')]

#Adding all VIP customers
Pref_VIP_Country = Pref_VIP_Country.append(VIP_Country)

#group by country, count by customerID and sort to see which country has more VIP+Preferred customers
Pref_VIP_Country.groupby(['Country'])[['CustomerID']].count().sort_values(by='CustomerID',ascending=False)

